# SemEval 2026 Task 8 - Task B: Generation

This notebook focuses on **Task B: Generation**. 
Goal: Given a question (and potentially contexts), generate an accurate answer.

**Output Format**:
The submission file must contain a `predictions` field (List of Objects), where each object has:
- `text`: The generated answer string.

In [ ]:
# --- KAGGLE SETUP ---
# Uncomment and run this cell FIRST if you are running on Kaggle.
# It clones the repo, installs dependencies, and sets the working directory.

# import os
# if not os.path.exists("llm-semeval-task8"):
#     !git clone https://github.com/LookUpMark/llm-semeval-task8.git

# %cd llm-semeval-task8
# !git checkout dev
# !pip install -c scripts/evaluation/constraints.txt -r scripts/evaluation/requirements.txt

In [ ]:
import os
import json
import sys
from tqdm import tqdm

# Locate Project Root (Robust for Local vs Kaggle)
if os.path.exists("src"):
    # We are in the root (e.g. Kaggle after %cd)
    project_root = os.getcwd()
else:
    # We are likely in 'notebooks/' (Local)
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.append(project_root)

try:
    from src.graph import app
except ImportError:
    print("ERROR: Check imports.")

In [ ]:
# --- CONFIGURATION ---
TEAM_NAME = "Gbgers"
TASK_TYPE = "TaskB"
# Adjust path based on execution environment
base_path = "." if os.path.exists("dataset") else ".."
INPUT_FILE = os.path.join(base_path, "dataset/human/generation_tasks/reference.jsonl")
OUTPUT_DIR = os.path.join(base_path, "data/submissions")
OUTPUT_FILE = os.path.join(OUTPUT_DIR, f"submission_{TASK_TYPE}_{TEAM_NAME}.jsonl")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Target: {OUTPUT_FILE}")

In [ ]:
with open(INPUT_FILE) as f:
    test_data = [json.loads(line) for line in f if line.strip()]
print(f"Loaded {len(test_data)} items.")

In [ ]:
results = []

print("Running Generation Task...")
for item in tqdm(test_data):
    question = item.get("question")
    
    try:
        # Invoke Graph
        # NOTE: If your graph supports 'documents' as input, you could pass 
        # item.get('contexts') here to do purely generation.
        # Otherwise, this runs Retrieval+Generation (which is also valid).
        resp = app.invoke({"question": question})
        gen_text = resp.get("generation", "")
        if not gen_text: gen_text = "No Answer"
    except Exception as e:
        print(f"Error on '{question}': {e}")
        gen_text = "Error"
    
    output_item = item.copy()
    # Task B requires 'predictions' list
    output_item["predictions"] = [{"text": gen_text}]
    
    results.append(output_item)

print(f"Done. Generated {len(results)} answers.")

In [ ]:
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for x in results:
        json.dump(x, f)
        f.write('\n')
print(f"Saved to {OUTPUT_FILE}")

# Validation
sample = results[0]
if "predictions" in sample and isinstance(sample["predictions"], list) and "text" in sample["predictions"][0]:
    print("\033[92mVALIDATION PASS: Structure correct.\033[0m")
else:
    print("\033[91mVALIDATION FAIL: 'predictions' formatting error.\033[0m")